In [342]:
class CareGiver:
    def __init__(self, name, id, trust, approved = False):
        self.m_name = name
        self.m_id = id
        self.m_trust = trust
        self.m_isApproved = approved
        self.m_ratings = {}
        self.m_recipient = []
        self.m_available = False
        self.m_ratingAvg = 0
        self.m_noOfClients = 0
        self.m_trust.m_careGivers.append(self)
        #bisect.insort_left(self.m_trust.m_careGivers, self)
    
    # Getters & Setters... 
    def GetTrusty(self):
        return self.m_trust
    def GetAvailability(self):
        return self.m_available
    def GetName(self):
        return self.m_name
    def GetAvgRating(self):
        return self.m_ratingAvg
    
    #for sorting
    def __lt__(self, other):
        return self.m_ratingAvg < other.m_ratingAvg
    
    def GetApproved(self):
        self.m_isApproved = True
        self.m_available = True
        return
    def SetAvailability(self, available):
        self.m_available = available
        return
        
    def AllocateARecipient(self):
        # Allocate a recipient if available, exchange connection, make recipient unavailable..
        if(len(self.m_recipient) < 4):
            recipient = self.m_trust.AllocateARecipient(self)
            print(self.m_name, " asked for the allocation of recipient..")
            if recipient != None :
                # care giver may or may not get the recipient now but, since he has made himself available, 
                #he could be picked by a recipient 
                self.m_recipient.append(recipient)
                print(self.m_name, " got the allocation of recipient.. : ", recipient.m_name)
                recipient.m_careGiver = self
                self.SetAvailability(False) 
            else: 
                print("Seems no recipient available at the moment..")
        else:
            print("Already have 4 recipients!!")

    def MakeYourselfAvailable(self):
        if len(self.m_recipient) < 4 :
            if self.m_isApproved :
                self.m_available = True
        # When caregiver is fired by the recipient, m_available is set to False, also when caregiver gets assigned new recipient
        # m_available is set to False. Hence, if the caregiver want's more recipient, he has to set m_available to True and also 
        # call the function AllocateARecipient()
        return
    
    def QuitARecipient(self, name, rating, reviews = ""):
        # we have to remove connections from both sides. Just the way we formed the connection on both sides...
        # remove the entry from the both sides 
        recipient = next((x for x in self.m_recipient if x.m_name == name), None)
        if(recipient != None):
            
            recipient.m_careGiver = None
            recipient.SetAvailability(True)
            recipient.AllotTheRating(rating, reviews)      
            self.m_recipient.remove(recipient)             ##can be generalized them in the trust class...
            print("Quit successfully...")
        else: 
            print("Not my recipient....")
        return 
        
        
    def AllotTheRating(self, rating, reviews):
        self.m_ratingAvg = (self.m_ratingAvg*self.m_noOfClients + rating)/(self.m_noOfClients + 1)
        self.m_ratings.update( {rating : reviews})
        self.m_noOfClients += 1
        print(self.m_name, " rating alloted")
        return None
        

In [343]:
 class Trust:
        def __init__(self):
            self.m_careGivers = []
            self.m_recipients = []
            
        def ShowAllTheCareGivers(self):
            return self.m_careGivers
        def AllocateARecipient(self, careGiver):
            self.m_recipients.sort(key=lambda x: x.m_ratingAvg, reverse=True)
            
            for rec in self.m_recipients :
                if(rec.GetAvailability()):
                    rec.m_available = False
                    return rec
            return None
        
        def AllocateACareGiver(self, recipient):
            self.m_careGivers.sort(key=lambda x: x.m_ratingAvg, reverse=True)
            for cg in self.m_careGivers :
                if(cg.GetAvailability()):
                    if(len(cg.m_recipient) < 4):
                        return cg
            return None
        
        def ShowAllCaregiversData(self):
            for cg in self.m_careGivers :
                if cg != None :
                    print(cg.GetName(), " rating : ", cg.m_ratingAvg)
                    for rating in cg.m_ratings :
                        print("\t\t\t", rating, " ", cg.m_ratings[rating])
            return 
        
        def ShowWhoIsTakingCareOfRecipients(self):
            for rp in self.m_recipients :
                if rp != None and rp.m_careGiver != None:
                    print(rp.GetName(), "    \t\t\t Care Giver : ", rp.m_careGiver.m_name)
                
            

In [344]:
class Recipient:
    
    def __init__(self, name, id, trust, available = True):
        self.m_name = name
        self.m_id = id;
        self.m_trust = trust
        self.m_available = available
        self.m_careGiver = None
        self.m_ratings = {}
        self.m_ratingAvg = 0
        self.m_noOfClients = 0
        self.m_trust.m_recipients.append(self)
        #bisect.insort_left(self.m_trust.m_recipients, self)
    
    # Getters & Setters...  
    def GetTrusty(self):
        return self.m_trust
    def GetAvailability(self):
        return self.m_available
    def GetName(self):
        return self.m_name
    def SetAvailability(self, available):
        self.m_available = available
        return
    #for sorting
    def __lt__(self, other):
        return self.m_ratingAvg < other.m_ratingAvg
        
    def AllocateACareGiver(self):
        # find recipient that is available and exchange connections and make recipient unavailable..
        if(self.m_careGiver == None):  
            careGiver = self.GetTrusty().AllocateACareGiver(self) 
            if(careGiver != None):
                # exchange info and make recipient unavailable. that's it....
                self.m_careGiver = careGiver
                careGiver.m_recipient.append(self)
                self.SetAvailability(False)
                print("Care giver allocated : ", careGiver.m_name)
            else: 
                    print(" No care givers available at the moment..")
        else:
            print("Already have a caregiver ! : ", self.m_careGiver.m_name) 
        
    def FireTheCareGiver(self, rating, reviews):
        # remove the caregiver info 
        # Make youself available... for next care giver
        if(self.m_careGiver != None):
            self.m_careGiver.m_recipient.remove(self) 
            self.SetAvailability(True)
            print(self.m_careGiver.m_name, " Just got fired !")
            self.m_careGiver.AllotTheRating(rating, reviews) 
            self.m_careGiver = None
        return
    
    def AllotTheRating(self, rating, reviews):
        self.m_ratingAvg = (self.m_ratingAvg*self.m_noOfClients + rating)/(self.m_noOfClients + 1)
        self.m_noOfClients += 1;
        self.m_ratings.update( {rating : reviews} )

In [345]:
theTrust = Trust()
c1 = CareGiver("Tushar", 1, theTrust)
c2 = CareGiver("Prashant", 2, theTrust)
c3 = CareGiver("Adam", 3, theTrust)
c4 = CareGiver("Scott", 4, theTrust)
r1 = Recipient("Al Paccino", 1, theTrust)
r2 = Recipient("Robert", 2, theTrust)
r3 = Recipient("Joe", 3, theTrust)
r4 = Recipient("Perry", 4, theTrust)
r5 = Recipient("Ross", 5, theTrust)
r6 = Recipient("Monica", 6, theTrust)

In [346]:
c1.GetApproved()
c2.GetApproved()
c3.GetApproved()
c4.GetApproved()

In [347]:
c1.AllocateARecipient()
c1.AllocateARecipient()

len(c1.m_recipient)

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Al Paccino
Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Robert


2

In [348]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Joe


In [349]:
c3.AllocateARecipient()

Adam  asked for the allocation of recipient..
Adam  got the allocation of recipient.. :  Perry


In [350]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Ross


In [351]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Monica


### Let's see connections so far

In [352]:
theTrust.ShowWhoIsTakingCareOfRecipients()

Al Paccino     			 Care Giver :  Tushar
Robert     			 Care Giver :  Tushar
Joe     			 Care Giver :  Prashant
Perry     			 Care Giver :  Adam
Ross     			 Care Giver :  Prashant
Monica     			 Care Giver :  Tushar


### When care givers quite on the recipients

In [353]:
c1.QuitARecipient("Monica", 4, "Good lady")

Quit successfully...


In [354]:
c3.QuitARecipient("Perry", 3, "Good lady")

Quit successfully...


In [355]:
c2.QuitARecipient("Ross", 1, "Good lady")

Quit successfully...


In [356]:
c1.QuitARecipient("Al Paccino", 1.5, "Huuaa")

Quit successfully...


In [357]:
c1.QuitARecipient("Robert", 2, "Good lady")

Quit successfully...


In [358]:
theTrust.ShowWhoIsTakingCareOfRecipients()

Joe     			 Care Giver :  Prashant


### Now the allocation of recipients will be according to their ratings 

In [359]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Monica


In [360]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Perry


In [361]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Robert


In [362]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Al Paccino


In [363]:
c1.AllocateARecipient()

Already have 4 recipients!!


In [364]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Ross


In [365]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Seems no recipient available at the moment..


In [366]:
theTrust.ShowWhoIsTakingCareOfRecipients()

Monica     			 Care Giver :  Tushar
Perry     			 Care Giver :  Tushar
Robert     			 Care Giver :  Tushar
Al Paccino     			 Care Giver :  Tushar
Ross     			 Care Giver :  Prashant
Joe     			 Care Giver :  Prashant


### When recipients fire a caregiver  

In [367]:
r6.AllocateACareGiver()

Already have a caregiver ! :  Tushar


In [368]:
r1.FireTheCareGiver(2, "Not good service")

Tushar  Just got fired !
Tushar  rating alloted


In [369]:
r2.FireTheCareGiver(3, "okkkkeeyyy")

Tushar  Just got fired !
Tushar  rating alloted


In [370]:
r4.FireTheCareGiver(4, "good enough")

Tushar  Just got fired !
Tushar  rating alloted


In [371]:
r5.FireTheCareGiver(5, "Excelante")

Prashant  Just got fired !
Prashant  rating alloted


In [372]:
c1.AllocateARecipient()

Tushar  asked for the allocation of recipient..
Tushar  got the allocation of recipient.. :  Perry


In [373]:
c2.AllocateARecipient()

Prashant  asked for the allocation of recipient..
Prashant  got the allocation of recipient.. :  Robert


In [374]:
c4.AllocateARecipient()

Scott  asked for the allocation of recipient..
Scott  got the allocation of recipient.. :  Al Paccino


In [378]:
c2.MakeYourselfAvailable()

In [380]:
r5.AllocateACareGiver()

Already have a caregiver ! :  Prashant


### Show all the connections and rating history   


In [381]:
theTrust.ShowWhoIsTakingCareOfRecipients()

Monica     			 Care Giver :  Tushar
Perry     			 Care Giver :  Tushar
Robert     			 Care Giver :  Prashant
Al Paccino     			 Care Giver :  Scott
Ross     			 Care Giver :  Prashant
Joe     			 Care Giver :  Prashant


In [382]:
theTrust.ShowAllCaregiversData()

Prashant  rating :  5.0
			 5   Excelante
Tushar  rating :  3.0
			 2   Not good service
			 3   okkkkeeyyy
			 4   good enough
Adam  rating :  0
Scott  rating :  0


### First Draft Done